In [1]:
import torch
from tqdm import tqdm
import os
from torch_sparse import SparseTensor
from tqdm import tqdm

In [5]:
import os.path

import torch
from torch_scatter import scatter
from torch_sparse import SparseTensor
from math import pi as PI
from rdkit import Chem
from rdkit.Chem import AllChem
from tqdm import tqdm
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def get_dist(pos,i,j):
    dist = (pos[i] - pos[j]).pow(2).sum(dim=-1).sqrt()
    return dist

def get_angle(pos,i,j,k):
        # Calculate angles. 0 to pi
    pos_ji = pos[i] - pos[j]
    pos_jk = pos[k] - pos[j]
    a = (pos_ji * pos_jk).sum(dim=-1) # cos_angle * |pos_ji| * |pos_jk|
    b = torch.cross(pos_ji, pos_jk).norm(dim=-1) # sin_angle * |pos_ji| * |pos_jk|
    angle = torch.atan2(b, a)
    return angle

def get_torsion(pos,k,i,j,t):
    pos_jt = pos[j] - pos[t]
    pos_ji = pos[j] - pos[i]
    pos_jk = pos[j] - pos[k]
    dist_ji = pos_ji.pow(2).sum(dim=-1).sqrt()
    plane1 = torch.cross(pos_ji, pos_jt)
    plane2 = torch.cross(pos_ji, pos_jk)
    a = (plane1 * plane2).sum(dim=-1) # cos_angle * |plane1| * |plane2|
    b = (torch.cross(plane1, plane2) * pos_ji).sum(dim=-1) / dist_ji
    torsion1 = torch.atan2(b, a) # -pi to pi
    torsion1 = torch.abs(torsion1)
    # torsion1[torsion1<=0]+=2*PI # 0 to 2pi
    return torsion1
def generate_dist_index(mol,valid=False):
    atom_num = mol.GetNumAtoms()
    start = torch.arange(atom_num).repeat_interleave(atom_num)
    end = torch.arange(atom_num).repeat(atom_num)
    if valid:
        valid_index = start != end
        start = start[valid_index]
        end = end[valid_index]
    return start,end
def generate_angle_index(mol,valid=False):
    i_all = []
    j_all = []
    k_all = []
    atoms  = [atom for atom in mol.GetAtoms()]
    for index in range(mol.GetNumAtoms()):
        atom = atoms[index]
        set1 = set([bond.GetBeginAtomIdx() for bond in atom.GetBonds()])
        set2 = set([bond.GetEndAtomIdx() for bond in atom.GetBonds()])
        group = list(set1 | set2)
        lth = len(group)
        if len(group) <= 2:
            continue
        i = torch.tensor(group).repeat_interleave(lth)
        j = torch.ones(lth**2,dtype=torch.int64) * index
        k = torch.tensor(group).repeat(lth)
        i_all.append(i)
        j_all.append(j)
        k_all.append(k)
    i_all = torch.cat(i_all)
    j_all = torch.cat(j_all)
    k_all = torch.cat(k_all)
    if valid:
        valid_index = (i_all != k_all) & (i_all != j_all) & (k_all != j_all)
        i_all = i_all[valid_index]
        j_all = j_all[valid_index]
        k_all = k_all[valid_index]
    return i_all,j_all,k_all

def generate_torsion_index(mol,valid=False):
    atoms  = [atom for atom in mol.GetAtoms()]
    group_dict = {}
    bond_start = [bond.GetBeginAtomIdx() for bond in mol.GetBonds()]
    bond_end = [bond.GetEndAtomIdx() for bond in mol.GetBonds()]
    for index in range(mol.GetNumAtoms()):
        atom = atoms[index]
        set1 = set([bond.GetBeginAtomIdx() for bond in atom.GetBonds()])
        set2 = set([bond.GetEndAtomIdx() for bond in atom.GetBonds()])
        group = list(set1 | set2)
        group_dict[index] = group
    i_all = []
    j_all = []
    k_all =[]
    t_all = []
    for atom_i,atom_j in zip(bond_start,bond_end):
        i_group = group_dict[atom_i]
        j_group = group_dict[atom_j]
        k = torch.tensor(i_group).repeat_interleave(len(j_group))
        t = torch.tensor(j_group).repeat(len(i_group))
        i = torch.ones(len(j_group)*len(i_group),dtype=torch.int64) * atom_i
        j = torch.ones(len(j_group)*len(i_group),dtype=torch.int64) * atom_j
        i_all.append(i)
        j_all.append(j)
        k_all.append(k)
        t_all.append(t)
    i_all = torch.cat(i_all)
    j_all = torch.cat(j_all)
    k_all = torch.cat(k_all)
    t_all = torch.cat(t_all)
    if valid:
        valid_index = (k_all != t_all) & (k_all != i_all) & (k_all != j_all) & (t_all != i_all) & (t_all != j_all)
        i_all = i_all[valid_index]
        j_all = j_all[valid_index]
        k_all = k_all[valid_index]
        t_all = t_all[valid_index]
    return k_all,i_all,j_all,t_all


In [7]:
d = "/home/zjh/mr/downstream/dataset/regression"
for p in os.listdir(d):

    dataset_path = os.path.join(d,p)
    if not dataset_path == "/home/zjh/mr/downstream/dataset/regression/FreeSolv.pth":
        continue
    if os.path.isdir(dataset_path):
        continue
    print(dataset_path)
    data = torch.load(dataset_path)

    smiles_list = []
    dist_list = []
    nmr_list = []
    angle_list = []
    torsion_list = []
    pos_list = []
    count =0
    label_list = []

    for row in tqdm(iter(data),total=len(data)):
        smiles = row[0]
        nmr = row[1]
        label = row[2]
        try:
            mol = AllChem.AddHs(Chem.MolFromSmiles(smiles))
            AllChem.EmbedMolecule(mol)
            AllChem.MMFFOptimizeMolecule(mol)
            pos=[]
            edge_index=[]
            num_nodes=len(mol.GetAtoms())
            for i in range(num_nodes):
                for j in range(i):
                    edge_index.append([i,j])
                x,y,z =mol.GetConformer().GetAtomPosition(i)
                pos.append([x,y,z])
            pos = torch.tensor(pos)
            dist_i,dist_j = generate_dist_index(mol,valid=True)
            angle_i,angle_j,angle_k = generate_angle_index(mol,valid=True)
            torsion_k,torsion_i,torsion_j,torsion_t = generate_torsion_index(mol,valid=True)
            dist = get_dist(pos,dist_i,dist_j)
            angle = get_angle(pos,angle_i,angle_j,angle_k)
            torsion = get_torsion(pos,torsion_k,torsion_i,torsion_j,torsion_t)

        except:
            print(smiles)
            continue

        if pos.shape[0] > 127:
            count+=1
            print("f{smiles} out 127")
            print(smiles)
            continue


        if (len(dist)==0 or len(angle)==0 or len(torsion)==0):
            print("valid of length")
            print(smiles)
            continue

        try:
            spt = SparseTensor(row=dist_i.long(),col=dist_j.long(),value=dist)
            dist_matrix = spt.to_dense()
        except:
            print("valid cannot covert adj dense")
            print(smiles)
            continue

        smiles_list.append(smiles)
        dist_list.append(dist_matrix)
        pos_list.append(pos)
        nmr_list.append(nmr)
        angle_list.append([angle_i.byte(),angle_j.byte(),angle_k.byte(),angle])
        label_list.append(label)

        torsion_list.append([torsion_k.byte(),torsion_i.byte(),torsion_j.byte(),torsion_t.byte(),torsion])
        count +=1
    print(f"{len(data) - count} is valid")
    base_dir = os.path.dirname(dataset_path)
    base_name = os.path.basename(dataset_path).split(".")[0]
    save_dir  = os.path.join(base_dir,base_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    smiles_save_path = os.path.join(save_dir,"smiles.pth")
    dist_save_path = os.path.join(save_dir,"dist.pth")
    nmr_save_path= os.path.join(save_dir,"nmr.pth")
    angle_save_path = os.path.join(save_dir,"angle.pth")
    torsion_save_path = os.path.join(save_dir,"torsion.pth")
    pos_save_path = os.path.join(save_dir,"pos.pth")
    label_save_path = os.path.join(save_dir,"label.pth")
    torch.save(smiles_list,smiles_save_path)
    torch.save(dist_list,dist_save_path)
    torch.save(nmr_list,nmr_save_path)
    torch.save(angle_list,angle_save_path)
    torch.save(torsion_list,torsion_save_path)
    torch.save(pos_list,pos_save_path)
    torch.save(label_list,label_save_path)

/home/zjh/mr/downstream/dataset/regression/FreeSolv.pth


  2%|▏         | 16/642 [00:00<00:13, 45.44it/s]

valid of length
C(Br)Br


  8%|▊         | 51/642 [00:01<00:10, 53.77it/s]

valid of length
C(F)(F)(F)F


...

valid of length
CBr
valid of length
CF


 70%|███████   | 452/642 [00:07<00:03, 57.31it/s]

valid of length
C(Cl)(Cl)(Cl)Cl


100%|██████████| 642/642 [00:11<00:00, 55.91it/s]


18 is valid


In [15]:
d = "/home/zjh/mr/downstream/dataset/regression"
for p in os.listdir(d):

    dataset_path = os.path.join(d,p)
    if dataset_path == "/home/zjh/mr/downstream/dataset/regression/FreeSolv.pth":
        continue
    if os.path.isdir(dataset_path):
        continue
    print(dataset_path)
    data = torch.load(dataset_path)

    smiles_list = []
    dist_list = []
    nmr_list = []
    angle_list = []
    torsion_list = []
    pos_list = []
    count =0
    label_list = []

    for row in tqdm(iter(data),total=len(data)):
        smiles = row[0]
        nmr = row[1]
        label = row[2]
        try:
            mol = AllChem.AddHs(Chem.MolFromSmiles(smiles))
            AllChem.EmbedMolecule(mol)
            AllChem.MMFFOptimizeMolecule(mol)
            pos=[]
            edge_index=[]
            num_nodes=len(mol.GetAtoms())
            for i in range(num_nodes):
                for j in range(i):
                    edge_index.append([i,j])
                x,y,z =mol.GetConformer().GetAtomPosition(i)
                pos.append([x,y,z])
            pos = torch.tensor(pos)
            dist_i,dist_j = generate_dist_index(mol,valid=True)
            angle_i,angle_j,angle_k = generate_angle_index(mol,valid=True)
            torsion_k,torsion_i,torsion_j,torsion_t = generate_torsion_index(mol,valid=True)
            dist = get_dist(pos,dist_i,dist_j)
            angle = get_angle(pos,angle_i,angle_j,angle_k)
            torsion = get_torsion(pos,torsion_k,torsion_i,torsion_j,torsion_t)

        except:
            print(smiles)
            continue

        if pos.shape[0] > 127:
            count+=1
            print("f{smiles} out 127")
            print(smiles)
            continue


        if (len(dist)==0 or len(angle)==0 or len(torsion)==0):
            print("valid of length")
            print(smiles)
            continue

        try:
            spt = SparseTensor(row=dist_i.long(),col=dist_j.long(),value=dist)
            dist_matrix = spt.to_dense()
        except:
            print("valid cannot covert adj dense")
            print(smiles)
            continue

        smiles_list.append(smiles)
        dist_list.append(dist_matrix)
        pos_list.append(pos)
        nmr_list.append(nmr)
        angle_list.append([angle_i.byte(),angle_j.byte(),angle_k.byte(),angle])
        label_list.append(label)

        torsion_list.append([torsion_k.byte(),torsion_i.byte(),torsion_j.byte(),torsion_t.byte(),torsion])
        count +=1
    print(f"{len(data) - count} is valid")
    base_dir = os.path.dirname(dataset_path)
    base_name = os.path.basename(dataset_path).split(".")[0]
    save_dir  = os.path.join(base_dir,base_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    smiles_save_path = os.path.join(save_dir,"smiles.pth")
    dist_save_path = os.path.join(save_dir,"dist.pth")
    nmr_save_path= os.path.join(save_dir,"nmr.pth")
    angle_save_path = os.path.join(save_dir,"angle.pth")
    torsion_save_path = os.path.join(save_dir,"torsion.pth")
    pos_save_path = os.path.join(save_dir,"pos.pth")
    label_save_path = os.path.join(save_dir,"label.pth")
    torch.save(smiles_list,smiles_save_path)
    torch.save(dist_list,dist_save_path)
    torch.save(nmr_list,nmr_save_path)
    torch.save(angle_list,angle_save_path)
    torch.save(torsion_list,torsion_save_path)
    torch.save(pos_list,pos_save_path)
    torch.save(label_list,label_save_path)

/home/zjh/mr/downstream/dataset/regression/Lipophilicity.pth


 11%|█         | 443/4200 [01:00<19:22,  3.23it/s]

f{smiles} out 127
CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(OC)[C@@H](O)[C@H](C)O2)[C@H](C)[C@@H](O[C@@H]3O[C@H](C)C[C@@H]([C@H]3O)N(C)C)[C@](C)(O)C[C@@H](C)\C(=N/OCOCCOC)\[C@H](C)[C@@H](O)[C@]1(C)O


 29%|██▉       | 1237/4200 [02:54<32:00,  1.54it/s]

f{smiles} out 127
CCn1c(C)c(C(=O)O)c(c2cccc(c2)N3CCN(CC3)c4ccc(NS(=O)(=O)c5ccc(N[C@H](CCN6CCC(O)CC6)CSc7ccccc7)c(c5)S(=O)(=O)C(F)(F)F)cc4)c1c8ccc(Cl)cc8


...

valid of length
C


 96%|█████████▌| 1084/1128 [00:42<00:01, 35.74it/s]

valid of length
ICI


100%|██████████| 1128/1128 [00:44<00:00, 25.28it/s]


13 is valid


In [16]:
d = "/home/zjh/mr/downstream/dataset/classify"
for p in os.listdir(d):

    dataset_path = os.path.join(d,p)
    if dataset_path == "/home/zjh/mr/downstream/dataset/regression/FreeSolv.pth":
        continue
    if os.path.isdir(dataset_path):
        continue
    print(dataset_path)
    data = torch.load(dataset_path)

    smiles_list = []
    dist_list = []
    nmr_list = []
    angle_list = []
    torsion_list = []
    pos_list = []
    count =0
    label_list = []

    for row in tqdm(iter(data),total=len(data)):
        smiles = row[0]
        nmr = row[1]
        label = row[2]
        try:
            mol = AllChem.AddHs(Chem.MolFromSmiles(smiles))
            AllChem.EmbedMolecule(mol)
            AllChem.MMFFOptimizeMolecule(mol)
            pos=[]
            edge_index=[]
            num_nodes=len(mol.GetAtoms())
            for i in range(num_nodes):
                for j in range(i):
                    edge_index.append([i,j])
                x,y,z =mol.GetConformer().GetAtomPosition(i)
                pos.append([x,y,z])
            pos = torch.tensor(pos)
            dist_i,dist_j = generate_dist_index(mol,valid=True)
            angle_i,angle_j,angle_k = generate_angle_index(mol,valid=True)
            torsion_k,torsion_i,torsion_j,torsion_t = generate_torsion_index(mol,valid=True)
            dist = get_dist(pos,dist_i,dist_j)
            angle = get_angle(pos,angle_i,angle_j,angle_k)
            torsion = get_torsion(pos,torsion_k,torsion_i,torsion_j,torsion_t)

        except:
            print(smiles)
            continue

        if pos.shape[0] > 127:
            count+=1
            print("f{smiles} out 127")
            print(smiles)
            continue


        if (len(dist)==0 or len(angle)==0 or len(torsion)==0):
            print("valid of length")
            print(smiles)
            continue

        try:
            spt = SparseTensor(row=dist_i.long(),col=dist_j.long(),value=dist)
            dist_matrix = spt.to_dense()
        except:
            print("valid cannot covert adj dense")
            print(smiles)
            continue

        smiles_list.append(smiles)
        dist_list.append(dist_matrix)
        pos_list.append(pos)
        nmr_list.append(nmr)
        angle_list.append([angle_i.byte(),angle_j.byte(),angle_k.byte(),angle])
        label_list.append(label)

        torsion_list.append([torsion_k.byte(),torsion_i.byte(),torsion_j.byte(),torsion_t.byte(),torsion])
        count +=1
    print(f"{len(data) - count} is valid")
    base_dir = os.path.dirname(dataset_path)
    base_name = os.path.basename(dataset_path).split(".")[0]
    save_dir  = os.path.join(base_dir,base_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    smiles_save_path = os.path.join(save_dir,"smiles.pth")
    dist_save_path = os.path.join(save_dir,"dist.pth")
    nmr_save_path= os.path.join(save_dir,"nmr.pth")
    angle_save_path = os.path.join(save_dir,"angle.pth")
    torsion_save_path = os.path.join(save_dir,"torsion.pth")
    pos_save_path = os.path.join(save_dir,"pos.pth")
    label_save_path = os.path.join(save_dir,"label.pth")
    torch.save(smiles_list,smiles_save_path)
    torch.save(dist_list,dist_save_path)
    torch.save(nmr_list,nmr_save_path)
    torch.save(angle_list,angle_save_path)
    torch.save(torsion_list,torsion_save_path)
    torch.save(pos_list,pos_save_path)
    torch.save(label_list,label_save_path)

/home/zjh/mr/downstream/dataset/classify/Ames.pth


 10%|█         | 676/6512 [00:42<10:41,  9.10it/s]

Cc1c(N)nc([C@H](CC(N)=O)NC[C@H](N)C(N)=O)nc1C(=O)N[C@H](C(=O)N[C@@H](C)[C@@H](O)[C@@H](C)C(=O)N[C@H](C(=O)NCCc1nc(-c2nc(C(=O)NCCCN[C@H](C)c3ccccc3)cs2)cs1)[C@@H](C)O)[C@@H](O[C@H]1O[C@@H](CO)[C@H](O)[C@@H](O)[C@@H]1O[C@@H]1O[C@H](CO)[C@@H](O)[C@H](OC(N)=O)[C@H]1O)c1c[nH]cn1


 13%|█▎        | 831/6512 [00:53<15:05,  6.27it/s]

Cc1c(N)nc([C@H](CC(N)=O)NC[C@@H](N)C(N)=O)nc1C(=O)N[C@@H](C(=O)N[C@H](C)[C@H](O)[C@H](C)C(=O)N[C@H](C(=O)NCCc1nc(-c2nc(C(=O)NCCCCN=C(N)N)cs2)cs1)[C@@H](C)O)[C@@H](O[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@@H]1O[C@@H]1O[C@H](CO)[C@@H](O)[C@H](OC(N)=O)[C@H]1O)c1c[nH]cn1


...

f{smiles} out 127
OC(C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C(NC(=O)C([NH3+])Cc1c2c([nH]c1)cccc2)Cc1c2c([nH]c1)cccc2)CO)CCC(=O)[O-])C(C)C)CC(=O)N)CC(C)C)CC(C(=O)NC(C(=O)NC(CCC(=O)[O-])C(=O)NC(Cc1ccccc1)C(=O)[O-])C)C


 14%|█▎        | 206/1513 [01:03<08:23,  2.60it/s]

f{smiles} out 127
OC(C(NC(=O)C(NC(=O)C(NC(=O)C([NH3+])CCC(=O)[O-])CC(C)C)CC(=O)[O-])CC(C)C)CC(C(=O)NC(C(C)C)C(=O)NC(CCC(=O)[O-])C(=O)NC(Cc1ccccc1)C(=O)[O-])C


100%|██████████| 1513/1513 [06:02<00:00,  4.17it/s]


0 is valid
